In [1]:
import streamlit as st
import langchain, os, openai, dotenv

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage



from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma

import chromadb
import dotenv, time

import os, glob
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

print(openai.__version__)



1.3.7


In [2]:
embeddings = AzureOpenAIEmbeddings(
    deployment=os.getenv("OPENAI_DEPLOYMENT_NAME_EMBED")
)

In [3]:
base_path = "./vectorDB/"

In [4]:
from langchain.document_loaders import DataFrameLoader
import pandas as pd

In [5]:
df = pd.read_parquet("TEIs.parquet.gzip")
print(len(df))
loader = DataFrameLoader(df[["article","text"]])
documents = []
documents.extend(loader.load())
text_splitter = CharacterTextSplitter(separator='\n',chunk_size=2000, chunk_overlap=100)
chunked_documents = text_splitter.split_documents(documents)

Created a chunk of size 2014, which is longer than the specified 2000
Created a chunk of size 2610, which is longer than the specified 2000
Created a chunk of size 2120, which is longer than the specified 2000
Created a chunk of size 2502, which is longer than the specified 2000
Created a chunk of size 2064, which is longer than the specified 2000
Created a chunk of size 2219, which is longer than the specified 2000
Created a chunk of size 2257, which is longer than the specified 2000
Created a chunk of size 2709, which is longer than the specified 2000
Created a chunk of size 2264, which is longer than the specified 2000
Created a chunk of size 3219, which is longer than the specified 2000
Created a chunk of size 2281, which is longer than the specified 2000
Created a chunk of size 2432, which is longer than the specified 2000
Created a chunk of size 2108, which is longer than the specified 2000
Created a chunk of size 2304, which is longer than the specified 2000
Created a chunk of s

327


Created a chunk of size 2071, which is longer than the specified 2000
Created a chunk of size 2865, which is longer than the specified 2000
Created a chunk of size 2512, which is longer than the specified 2000
Created a chunk of size 3283, which is longer than the specified 2000
Created a chunk of size 2030, which is longer than the specified 2000
Created a chunk of size 2902, which is longer than the specified 2000
Created a chunk of size 2784, which is longer than the specified 2000
Created a chunk of size 2803, which is longer than the specified 2000
Created a chunk of size 2034, which is longer than the specified 2000
Created a chunk of size 2061, which is longer than the specified 2000
Created a chunk of size 2175, which is longer than the specified 2000
Created a chunk of size 2002, which is longer than the specified 2000
Created a chunk of size 2130, which is longer than the specified 2000
Created a chunk of size 2097, which is longer than the specified 2000
Created a chunk of s

327 6706


In [9]:
print(len(documents),len(chunked_documents))

327 6706


In [8]:
if not os.path.isfile(base_path+"chroma.sqlite3"):
    print("Start a new DB")
    vectordb = Chroma.from_documents(
        documents=[chunked_documents[0]],
        embedding=embeddings,
        persist_directory=base_path
    )
    vectordb.persist()
else:
    print("Continue on the DB")
    vectordb = Chroma(persist_directory=base_path,embedding_function=embeddings)
    print(len(len(vectordb.get()["ids"])),"elements already stored.")
    for doc in chunked_documents:
        # Check if the text already exists somewhere
        if not len(vectordb.get(where_document={"$contains":doc.page_content})["documents"]):
            vectordb.add_documents(
                documents=[doc], 
                embedding=embeddings, 
                persist_directory=base_path
            )
            # Ugly hack to avoid reaching token per min limit 
            # So it sleeps 1s between page
            time.sleep(0.2)
            vectordb.persist()


Continue on the DB


KeyboardInterrupt: 

In [ ]:
def create_agent_chain(llm):
    chain = load_qa_chain(llm, chain_type="stuff")
    return chain

In [ ]:
def get_llm_response(query,vectordb,temperature=0.1,k=10):

    llm = AzureChatOpenAI(
        deployment_name=os.getenv("OPENAI_DEPLOYMENT_NAME"),
        temperature=temperature,
        openai_api_version="2023-05-15"
    )
    chain = create_agent_chain(llm)
    matching_docs = vectordb.similarity_search(query,k)
    answer = chain.run(input_documents=matching_docs, question=query)
    return answer, matching_docs

In [ ]:
answer, matching_docs = get_llm_response("What are the different vectors of contagious diseases in cities?",vectordb,temperature=0.2,k=10)

In [ ]:
print(answer)

The text mentions several vectors of contagious diseases in cities, including:

- Mosquitoes: Aedes aegypti and Aedes albopictus are urban mosquitoes that spread diseases such as dengue fever, Zika, and chikungunya. They proliferate in manmade containers like tires, pots, and water storage containers.

- Triatomine bugs: These bugs transmit Chagas disease and are present in urban centers in Brazil due to rural-urban human migration.

- Sandflies: Sandflies transmit diseases like leishmaniasis and can be found in urban communities in northwest Argentina.

- Tsetse flies: Tsetse flies transmit diseases like human African trypanosomiasis (HAT) and are implicated in vector-borne diseases.

- Fleas: Fleas can transmit diseases and are considered vectors in certain contexts.

- Black flies: Black flies are also vectors of diseases in some regions.

- Aquatic snails: Aquatic snails are implicated in the transmission of diseases like schistosomiasis.

It's important to note that different dise

In [ ]:
matching_docs

[Document(page_content='. Electric pumps installed as part of a rural electrification project in Brazil were found to be a risk factor for schistosomiasis spread\n. The urban mosquitos Aedes aegypti and Aedes albopictus that spread dengue fever, Zika, and chikungunya proliferate in manmade containers, such as tyres, pots and water storage containers\n. These vectors proliferate in long occupied urban spaces where impoverished people live in crowded conditions; major epidemics of dengue continue to occur in the Favelas of Latin America\n.\nThe exact ways in which urbanization and vector densities interact vary. Levy et al.\nexplored changes in the built environment of a major city in Peru, and found that Chagas disease (influenced by housing quality) was most prevalent among older (and slightly more wealthy) sections of the city due to land tenure security, which drove residents to invest more in their housing and perpetuate Triatoma infestans colonies. Visceral leishmaniasis (VL), gene